# 実用上のモデルチューニング
本単元からは、実務でモデルのチューニングのために使われることの多いpytorchの様々な機能について紹介する。  
ここまでの単元では扱いきれなかった、関数の詳細な使い方や、重要な活性化関数や損失関数、最適化関数を紹介する。  

## この単元の目標
- 様々な活性化関数を知ろう
- 様々な損失関数を知ろう
- 様々な最適化関数を知ろう

## 1. 活性化関数によるモデルチューニング
「relu関数」はすでに紹介済みだが、有名な活性化関数は他に「sigmoid関数」や「softmax関数」がある。  
pytorchで実装されている、これらの活性化関数の使い方と挙動を復習も兼ねて確認しよう。  

<br>

**【relu関数】**  
relu関数は、**「0未満の値を0に変換する」**という関数だ。0以上の値に対しては特に作用しない。  
pytorchでは、`torch.nn.functional`モジュールで`relu()`関数として実装されている。  
<br>

**【softmax関数】**  
softmax関数は、**「入力ベクトルの要素が0以上で、和が1になるようにスケール処理する」**という関数だ。  
スケール処理なので、**要素ごとの大小関係は変わらない。**  
使用例としては、MLPの出力層の出力にsoftmax関数を適用することで、その出力ベクトルは「モデルが予測した各クラスごとの確率」と言える。  
pytorchでは`torch.nn.fuctional`モジュールで`softmax()`関数として実装されている。  
本講座で扱ったような、**1次元のベクトルにsoftmax関数を適用する場合は、引数`dim`に`0`を指定する。**  
`dim=2`以上の場合は複雑で、今すぐ必要な知識ではないので、説明は割愛する。  

<br>

**【sigmoid関数】**  
「sigmoid」とかいて「シグモイド」と読む。  
**「入力値を0から1に収まるように処理する」**という関数だ。  
単調増加な関数で、入力値が小さいほど出力値は0に近くなり、大きいほど1に近くなる。  
pytorchでは`torch`モジュールで`sigmoid()`関数として使用できる。  
`torch.nn.functional`モジュールでも呼び出すことはできるが、公式的に前者を推奨されているため、そちらをおすすめする。



【例題】 以下のコードを実行して、それぞれの活性化関数の挙動を確認する。

In [1]:
import torch
from torch.nn import functional as F

x = torch.tensor([-2.0, 1.0, 4.0, 0.0])

print('relu:', F.relu(x))
print('softmax:', F.softmax(x, dim=0))
print('sigmoid:', torch.sigmoid(x))

relu: tensor([0., 1., 4., 0.])
softmax: tensor([0.0023, 0.0465, 0.9341, 0.0171])
sigmoid: tensor([0.1192, 0.7311, 0.9820, 0.5000])


- ```
relu: tensor([0., 1., 4., 0.])
softmax: tensor([0.0023, 0.0465, 0.9341, 0.0171])
sigmoid: tensor([0.1192, 0.7311, 0.9820, 0.5000])
```
と表示されていれば成功だ。
- relu()関数の結果、0以下の値が全て0となっている。
- softmax()関数の結果、値の大小関係は変わらずに、全ての値が正で、和が1となっている。
- sigmoid()関数の結果、値の大小関係は変わらずに、全ての値が0から1に収まっている。  
ちなみに、0を入力したときの出力は0.5となる。

【問題】 テンソル`[-5.0, 5.0, -10.0, 10.0]`に対して、3つの活性化関数を適用して出力を表示しよう。

In [2]:
import torch
from torch.nn import functional as F

x = torch.tensor([-5.0, 5.0, -10.0, 10.0])

print('relu:', F.relu(x))
print('softmax:', F.softmax(x, dim=0))
print('sigmoid:', torch.sigmoid(x))

relu: tensor([ 0.,  5.,  0., 10.])
softmax: tensor([3.0385e-07, 6.6928e-03, 2.0474e-09, 9.9331e-01])
sigmoid: tensor([6.6929e-03, 9.9331e-01, 4.5398e-05, 9.9995e-01])


- ```
relu: tensor([ 0.,  5.,  0., 10.])
softmax: tensor([3.0385e-07, 6.6928e-03, 2.0474e-09, 9.9331e-01])
sigmoid: tensor([6.6929e-03, 9.9331e-01, 4.5398e-05, 9.9995e-01])
```
と表示されていれば成功だ。
- relu()関数の結果、0以下の値が全て0となっている。
- softmax()関数の結果、値の大小関係は変わらずに、全ての値が正で、和が1となっている。
- sigmoid()関数の結果、値の大小関係は変わらずに、全ての値が0から1に収まっている。  

## 2. 損失関数によるモデルチューニング
次に扱うのは、損失関数だ。  
損失関数は、クラスとして実装されているものをインスタンスで宣言したのち、  
損失計算するタイミングで`__call__()`関数を呼び出す方法が一般的だ。  
さて、頻繁に用いられる損失関数`CrossEntropyLoss()`関数を新たに加えた、様々な損失関数を学ぼう。  

**【nn.MSELoss】**  
MSELossは最も主流な損失関数の一つだ。  
2つのベクトルの**要素ごとの差の2乗**から誤差を算出する。  
pytorchでは、`torch.nn`モジュールで`MSELoss()`としてクラスが定義されている。  
インスタンス宣言時に、引数`reduction='mean'`で**要素ごとの誤差の平均**、`reduction='sum'`で**合計**を損失値として出力する。  
`reduction='none'`を指定すると、平均や合計処理をする前の、**要素ごとの誤差をテンソルとして**出力する。  

**【nn.CrossEntropyLoss】**  
CrossEntropyLoss（以下：CE）もかなり頻繁に使用される損失関数で、  
MSEが「全要素の誤差」を求めていたのに対し、こちらは**「正解ラベルと対応する出力との差のみ」**を計算する。
しかし、正しくこれを用いるためには、**モデルの出力をsoftmax関数などで確率にする必要**がある。  
なぜならば、正解ラベルに対するモデルの出力が「1」のときに損失が0となってしまうからだ。  

以下の例を見てほしい。  
【例：正解ラベルが「1」（one-hotベクトルで`[0, 1, 0]`）  】 →**モデルの出力ベクトルの2番目の要素と比較する**
- モデルの出力: `[0.2 0.2 0.6]`（確率ベクトル） → 損失=**0.2と1との差**
- モデルの出力: `[1.0 1.0 3.0]`（確率ベクトルでない） → 損失=**1と1との差**

MSEと違いCEは、正解ラベルと対応する出力の誤差 **"のみ"** を計算する。  
2つ目の場合では、「1と1との損失」を計算するが、当然ながらこれは**「0」**となってしまい、これ以上最適化できない。  
つまり、正解ラベルとの比較対象が確率ベクトルになっていないと、**正しく損失が計算されない**可能性がある。  
定義式など、CEの詳細な説明はここでは割愛するが、**「モデルの出力を確率ベクトルに変換する必要がある」**ということは覚えておこう。  

CEを使用する上で確率値に変換しておく必要があるが、pytorchでは確率値に変換する`softmax()`を組み込んだ`nn.CrossEntropyLoss()`としてクラス定義されている。 

このため、CEを使用する際にわざわざsoftmax関数で確率値に変換する必要はない。

`nn.MSE()`と同様に`({予測値}, {正解ラベル})`という形式で引数を与えるが、  
正解ラベルは**one-hotベクトルではなく、正解ラベルを表す値をそのまま与える。**

さらに，CEの派生でBinaryCrossEntropyLossというものがある。  
2クラス分類用に定義された損失関数で、pytorchには`nn.BCELoss`として実装してある。  
詳細は、余裕があればぜひ調べてみてほしい。  

【例題】 以下のコードを実行して、それぞれの損失関数の挙動を確認する。

In [16]:
from torch import nn

# バッチサイズ=1と見立てたデータを用意
x = torch.tensor([[2., 5., 3.]])
mse_label = torch.tensor([[0, 1, 0]])
cel_label = torch.tensor([1])

print('MSE:', nn.MSELoss(reduction='mean')(x, mse_label))  ## インスタンス宣言と__call__()を同時に呼び出している
print('CrossEntropy:', nn.CrossEntropyLoss()(x, cel_label))

MSE: tensor(9.6667)
CrossEntropy: tensor(0.1698)


- ```
MSE: tensor(0.1267)
CrossEntropy: tensor(0.9398)
```
と表示されていれば成功だ。
- 異なる種類の損失値は、比較できないので注意しよう。  
この場合、`x`も正解データも同じものを与えているが「CE>MSE」となっている。

【問題】 テンソル`x = torch.tensor([[0.3, 0.3, 0.3, 0.1]])`に対して正解ラベルを3としてMSEとCEを使って損失を計算しよう。

In [19]:
from torch import nn

# バッチサイズ=1と見立てたデータを用意
x = torch.tensor([[0.3, 0.3, 0.3, 0.1]])
mse_label = torch.tensor([[0, 0, 0, 1]])
ce_label = torch.tensor([3])

print('MSE:', nn.MSELoss(reduction='mean')(x, mse_label))  ## インスタンス宣言と__call__()を同時に呼び出している
print('CrossEntropy:', nn.CrossEntropyLoss()(x, ce_label))

MSE: tensor(0.2700)
CrossEntropy: tensor(1.5399)


- ```
MSE: tensor(0.2700)
CrossEntropy: tensor(1.5399)
```
と表示されていれば成功だ。
- 例題と同じように「CE>MSE」となっているが、常に「CE>MSE」と言う訳ではない。

## 3. 最適化関数によるモデルチューニング
最後は、最適化関数について学ぼう。  
最適化関数も損失関数と同様に、一般的には「インスタンスを作成→`__call__()`を呼び出す」という流れで実行する。  

**【SGD】**  
最適化関数の説明をするときに、一番イメージしやすくシンプルなのは**SGD**だろう。  
SGDの正式名称は「Stochastic Gradient Descent」といい、日本語では**「確率的勾配降下法」**と言う。  
「勾配降下法」は「逆伝播」の単元で説明した通り、勾配から損失を最小化するようにパラメータを更新する事だが、SGDは「確率的」にこれを行う。  
何が確率的かというと、勾配を全データに対してではなく**ランダムサンプルしたデータに対して計算し、最適化を行う**のだ。  
全データに対して計算される勾配は一意に定まるが、**ランダムサンプルした場合の勾配はデータによって変化する**性質を利用している。  
pytorchでは、`torch.optim`モジュールの`SGD()`としてクラスが定義されている。  
学習率は、自動で設定されないので、**引数`lr`で明示的に指定する必要がある。**

**【Momentum】**  
**Momentum**は、SGDを改良した最適化関数で、日本語で「運動量」や「勢い」と訳す。  
その名の通り、SGDに現在の勾配だけでなく**過去の勾配情報を追加**させることで、**「勢い」を考慮した最適化**を行う。  
pytorchでは、`Momentum()`というクラスはなく、`SGD()`の引数`momentum`を指定することで宣言できる。  
引数`momentum`は標準で`0（勢いを考慮しない）`で、使用する場合はこれに数値を指定する。  
例えば、前回の勾配を半分だけ考慮したい場合には`0.5`、1割だけ考慮したい場合は`0.1`だ。  
下式を見てもらうとわかるが、`momentum>1`だと過去の勾配をより強く考慮してしまうため正しく最適化できない可能性が高い。

今回の勾配を $ d_t $　、前回の勾配を $ d_{t-1} $ として、

$$ 勾配を求める式: (勾配) = d_t + momentum × d_{t-1} $$

**【Adam】**  
新しい手法ではないが、現在でも主流な最適化関数が**Adam**だ。  
Adamは、Momentumに、**「勾配の大きさに応じて学習率`lr`を調節する機能」**を追加した最適化関数だ。  
今回紹介している3つの最適化関数は、SGD→Momentum→Adamという順に進化している。  
ご存知の通り、pytorchでは`torch.optim`モジュールに`Adam()`としてクラスが定義してある。  
`Adam()`には、`lr`の他に`betas`、`eps`、`weight_decay`、`amsgrad`といった様々なハイパーパラメータが引数として指定できる。  
かなり難しい内容なのでここでは扱わないが、興味があれば調べてみてほしい。

【例題】 以下のコードを実行して、それぞれの最適化関数を宣言する。

In [21]:
from torch import optim
from torchvision.models import vgg11

model = vgg11()

adam = optim.Adam(model.parameters())
sgd = optim.SGD(model.parameters(), lr=0.01)
momentum = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

print('adam:',adam)
print('sgd:',sgd)
print('momentum:',momentum)


adam: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
sgd: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)
momentum: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)


- ```
adam: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
sgd: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)
momentum: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
```
と表示されていれば成功だ。

【問題】 例題のコードから、3つの最適化関数の引数`lr`を`0.2`に変更して宣言し、例題と同じように出力してみよう。  
第1引数のモデルのパラメータは`vgg11()`のものを使おう。

In [22]:
model = vgg11()

adam = optim.Adam(model.parameters(), lr=0.2)
sgd = optim.SGD(model.parameters(), lr=0.2)
momentum = optim.SGD(model.parameters(), lr=0.2, momentum=0.9)

print('adam:',adam)
print('sgd:',sgd)
print('momentum:',momentum)


adam: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.2
    weight_decay: 0
)
sgd: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.2
    momentum: 0
    nesterov: False
    weight_decay: 0
)
momentum: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.2
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)


- ```
adam: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.2
    weight_decay: 0
)
sgd: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.2
    momentum: 0
    nesterov: False
    weight_decay: 0
)
momentum: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.2
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
```
と表示されていれば成功だ。
- 例題の出力と比較して、`lr`の値が`0.2`と変わっていることを確認しよう。